In [1]:
using HTTP #Exporto las funciones
using JSON
using pandas

#Pido los datos a la api
url = "https://api.n2yo.com/rest/v1/satellite/tle/25544&apiKey=T58NBR-Q5NHQE-Y76PS5-59V1"
res = HTTP.get(url)
json = JSON.parse(String(res.body))
puntos_satelites = json["tle"]

data_csv = Pandas.read_csv("points.csv")
print(data_csv)

[ Info: Installing pandas via the Conda pandas package...
[ Info: Running `conda install -y pandas` in root environment


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done




==> WARNING: A newer version of conda exists. <==
    current version: 24.3.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda





## Package Plan ##

  environment location: C:\Users\ElAdagioDeJP\.julia\conda\3\x86_64

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-2.2.2               |  py310hb4db72f_1        11.3 MB  conda-forge
    python-dateutil-2.9.0      |     pyhd8ed1ab_0         218 KB  conda-forge
    python-tzdata-2024.1       |     pyhd8ed1ab_0         141 KB  conda-forge
    pytz-2024.1                |     pyhd8ed1ab_0         184 KB  conda-forge
    six-1.16.0                 |     pyh6c4a22f_0          14 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.8 MB

The following NEW packages will be INSTALLED:

  pandas             conda-forge/win-64::pandas-2.2.2-py310hb4db72f_1 
  python-dateutil    conda-forge/noarch::python-dateutil-2.9.0-pyhd8ed1ab_0 
  python-t